# Analytic modelling

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import factorial

## Input variables

In [3]:
n = 2
lambda0 = 0.7
p0 = np.array([1, 0])
p = np.array([[0.1, 0.9], [0.2, 0.4]])
mu = np.array([1/0.4, 1/0.6])
r = np.array([2, 3])

## Finding $\lambda$ by solving system of linear equations $\lambda_{i}=\sum_{j=0}^{n}P_{ji}\lambda_{j}$

In [4]:
lambdas = np.linalg.solve(np.transpose(p) - np.identity(n), -lambda0 * p0)

## Finding $e_{i}$ using $\lambda_{i} = e_{i} \lambda_{0}$ 

In [5]:
e = lambdas/lambda0

## Checking stationarity of the system using criteria $\lambda_{i} < \mu_{i} r_{i}$

In [6]:
assert((lambdas < mu * r).all())

## Calculating normalizing coefficient using 
$C = \left(\left(\frac{\lambda}{\mu}\right)^r \frac{1}{r! \left(1 - \frac{\lambda}{\mu r}\right)} + \sum_{k=0}^{r-1} \left(\frac{\lambda}{\mu}\right)^k \frac{1}{k!}\right)^{-1}$

In [7]:
c = np.zeros((n, ))

for i in range(n):
    c[i] = (lambdas[i]/mu[i]) ** r[i] / (np.math.factorial(r[i]) * (1 - lambdas[i]/(mu[i] * r[i])))
    for k in range(r[i]):
        c[i] += (lambdas[i]/mu[i])**k / np.math.factorial(k)

c = 1 / c

In [8]:
pr = []
maxn = 20
eps = 1e-5

for i in range(n):
    pri = 0
    pr.append([])
    
    for k in range(maxn):
        prk = (lambdas[i]/mu[i])**k * c[i]
        
        if k > r[i]:
            prk /= (np.math.factorial(r[i]) * (r[i]**(k - r[i])))
        else:
            prk /= np.math.factorial(k)
        
        pr[i].append(prk)
        pri += prk

    assert(abs(pri - 1) < eps)
pr = np.array(pr)

## Calculating efficiency parameters for the system 
$L_{i} = \sum_{j=r+1}^{\infty} (j-r) P_{i}(j)$ - average queue length for node i

$R_{i} = \frac{\lambda_{i}}{\mu_{i}}$ - average number of channels in use in node i

$M_{i} = L_{i} + R_{i}$ - average number of requests in node i

$Q_{i} = \frac{L_{i}}{\lambda_{i}}$ - average waiting time in queue i

$T_{i} = \frac{M_{i}}{\lambda_{i}}$ - average time spent in node i by request 

$T = \sum_{i=1}^{n} e_{i} T_{i}$ - average time spent in system by request 

In [9]:
L = np.zeros((n, ))
for i in range(n):
    for k in range(r[i] + 1, maxn):
        L[i] += (k - r[i])* pr[i, k]

In [10]:
R = lambdas/mu
M = L + R 
Q = L / lambdas
T = M / lambdas
T_all = sum(e * T)

## Output variables

In [12]:
print ("L = ", L)
print ("R = ", R)
print ("M = ", M)
print ("Q = ", Q)
print ("T = ", T)
print ("T_all = ", T_all)

L =  [0.02687035 0.05515029]
R =  [0.46666667 1.05      ]
M =  [0.49353702 1.10515029]
Q =  [0.02303173 0.03151445]
T =  [0.42303173 0.63151445]
T_all =  2.2838390107372395
